In [1]:
from __future__ import print_function

import cobra
import cobra.test
# import mackinac
import numpy as np
import csv
import glob
import pickle
import pandas as pd
import time
import sys
from copy import deepcopy
from collections import defaultdict
# from cobra.flux_analysis import gapfill
from gapfill2 import *
from cobra.flux_analysis import pfba
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.flux_analysis import sample
from cobra.core.solution import get_solution


# Set default logger to python logger to avoid warnings given when adding reactions and/or metaboites 
# because "cobra.core.model" doesn't innately have a logger.
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

In [2]:
def set_media(model, media, universal, verbose=False):

    # Find and close all exchange reactions in the model
    model_rxns = [rxn.id for rxn in model.reactions]
    for rxn in model_rxns:
        if rxn.startswith('EX_') and rxn.endswith('_e'):
            model.reactions.get_by_id(rxn).lower_bound = 0.0

    # Check for existence of exchange reactions for the media metabolites in the model
    for metabolite in media:
        met = metabolite[1]+'_e'
        if 'EX_'+met in model_rxns:
            model.reactions.get_by_id('EX_'+met).lower_bound = -1000.
        else:
            # Create exchange reaction and add to model
            if verbose:
                print("added exchange rxn for " + met)
            new_exchange = cobra.Reaction('EX_'+met)
            new_exchange.name = met + ' exchange'
            met_obj = universal.metabolites.get_by_id(met)
            new_exchange.add_metabolites({met_obj:-1})
            new_exchange.lower_bound = -1000.
            new_exchange.upper_bound = 1000.
            model.add_reaction(new_exchange)
            model.repair()

In [3]:
# Basal Synthetic Media
bsm = [
    ['H+','cpd00067'],
    ['H2O','cpd00001'],
    ['CO2','cpd00011'],
    ['O2','cpd00007'],
    ['N2','cpd00528'], 
#     ['H2','cpd11640'], # Only with no O2
    
    ['K+','cpd00205'],
    ['Na+','cpd00971'],
    ['Mg','cpd00254'],
    ['Mn2+','cpd00030'],
    ['Fe2+','cpd10515'], # Iron ion in heme
    ['Ca2+','cpd00063'], # Calcium pantothenate;cpd19112
    
    ['Vitamin B12r','cpd00423'], # C62H91CoN13O14P : cobalamin;cpd03424;cpd00730 : not present in any exchange reactions
    ['Cobinamide','cpd03422'], #EXs : related to cobalamin (B12) Added to ensure cells have access to B12
    ['BIOT','cpd00104'], # C10H15N2O3S : biotin B7
    ['PAN','cpd00644'], # C9H16NO5 : Pantothenate B5
    ['Folate','cpd00393'], # C19H17N7O6 : B9
    ['Niacin','cpd00218'], # C6H4NO2 : B3
    ['Pyridoxal','cpd00215'], # C8H9NO3 : B6
    ['Riboflavin','cpd00220'], # C17H19N4O6 : B2
    ['thiamin','cpd00305'], # C12H17N4OS : B1
    
#     ['Phosphate','cpd00009'], # HO4P : In M9 Defaults
    
    ['Thioglycolate','cpd01415'], # C2H3O2S : not present in any exchange reactions
#     ['Sulfate','cpd00048'], # O4S : In M9 Defaults
    
    ['Acetate','cpd00029'], # C2H3O2 : not present in any exchange reactions
    ['Citrate','cpd00137'], # C6H5O7 : Consider removing. 
#     ['Polysorbate 60','cpd24450'], # C35H68O10 : Almost tween 80 : not present in any reactions
#     ['Ethyl acetate','cpd00633'], # C4H8O2 : not present in any exchange reactions, only present in one reaction at all
    
    ['ABEE','cpd00443'] # C7H6NO2 : aminobenzoate : not present in any exchange reactions
]

# Potentially add to BSM (from M9 media)
M9_ions = [
    ['Cl-','cpd00099'],
    ['Co2+','cpd00149'],
    ['Cu2+','cpd00058'],
    ['Fe3','cpd10516'],
#     ['Sodium molybdate','cpd11145'], # This doesn't connect to anything
    ['Ni2+','cpd00244'],
    ['Selenate','cpd03396'],
    ['Selenite','cpd03387'],
    ['Zn2+','cpd00034']
]

# Enviromental Metabolites with Exchange reactions
[
#     ['CO2','cpd00011'], #EXs : 
#     ['Ca2+','cpd00063'], #EXs : 
#     ['Cd2+','cpd01012'], #EXs : Removed because toxic
#     ['chromate','cpd11595'], #EXs : Removed because toxic
#     ['Cl-','cpd00099'], #EXs : 
#     ['Co2+','cpd00149'], #EXs : In M9
#     ['Cu2+','cpd00058'], #EXs : In M9
#     ['Fe2+','cpd10515'], #EXs : 
#     ['H+','cpd00067'], #EXs : 
#     ['H2','cpd11640'], #EXs : 
#     ['H2O','cpd00001'], #EXs : 
#     ['Hg2+','cpd00531'], #EXs : Removed because toxic
#     ['K+','cpd00205'], #EXs : 
#     ['Mg','cpd00254'], #EXs : 
#     ['Mn2+','cpd00030'], #EXs : 
#     ['Na+','cpd00971'], #EXs : 
#     ['Ni2+','cpd00244'], #EXs : In M9
#     ['O2','cpd00007'], #EXs : 
#     ['Pb','cpd04097'], #EXs : Removed because toxic
#     ['Zn2+','cpd00034'], #EXs : In M9
#     ['fe3','cpd10516'] #EXs : In M9
]

# M9 Base : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4932939/
# [
#     ['Ca2+','cpd00063'],
#     ['Cl-','cpd00099'],
#     ['CO2','cpd00011'],
#     ['Co2+','cpd00149'],
#     ['Cu2+','cpd00058'],
#     ['Fe2+','cpd10515'],
#     ['Fe3','cpd10516'],
#     ['H+','cpd00067'],
#     ['H2O','cpd00001'],
#     ['K+','cpd00205'],
#     ['Mg','cpd00254'],
#     ['Mn2+','cpd00030'],
#     ['Sodium molybdate','cpd11145'],
#     ['Na+','cpd00971'],
#     ['Ni2+','cpd00244'],
#     ['Selenate','cpd03396'],
#     ['Selenite','cpd03387'],
#     ['Zn2+','cpd00034']
# ]

# M9 default carbon, nitrogen, phosphorous, and sulfur sources
M9_sources = [
    ['D-Glucose','cpd00027'],
    ['NH3','cpd00013'], # this is actually NH4 : ammonium
    ['Phosphate','cpd00009'],
    ['Sulfate','cpd00048']
]

# Vitamins
vit_k = [
#     ['BIOT','cpd00104'], #EXs : Biotin
#     ['Cobinamide','cpd03422'], #EXs : related to cobalamin (B12)
#     ['Folate','cpd00393'], #EXs : 
    ['Menaquinone 7','cpd11606'], #EXs : Vitamine K2 : Add when there is no O2
#     ['Niacin','cpd00218'], #EXs : 
#     ['PAN','cpd00644'], #EXs : Pantothenate
#     ['Pyridoxal','cpd00215'], #EXs : 
#     ['Riboflavin','cpd00220'], #EXs : 
#     ['Thiamin','cpd00305'] #EXs : 
]

# For aerobic simulations, O2 was added with a lower bound of −20 and to 0 for anaerobic simulations.

# DNA/RNA related metabolites
rna_bases = [
#     ['35ccmp','cpd00696'], #EXs : 
#     ['AMP','cpd00018'], #EXs : 
    ['Adenosine','cpd00182'], #EXs : In BSM (as adenine)
#     ['Adenosine 3-5-bisphosphate','cpd00045'], #EXs : 
    ['Cytosine','cpd00307'], #EXs : 
#     ['Deoxyadenosine','cpd00438'], #EXs : 
#     ['Deoxycytidine','cpd00654'], #EXs : 
#     ['Deoxyguanosine','cpd00277'], #EXs : In BSM
#     ['Deoxyinosine','cpd03279'], #EXs : 
#     ['Deoxyuridine','cpd00412'], #EXs : 
#     ['GMP','cpd00126'], #EXs : 
#     ['GTP','cpd00038'], #EXs : 
    ['Guanosine','cpd00311'], #EXs : In BSM (as Guanine)
#     ['Inosine','cpd00246'], #EXs : 
#     ['HYXN','cpd00226'], #EXs : Hypoxanthine
#     ['Nicotinamide ribonucleotide','cpd00355'], #EXs : 
#     ['TTP','cpd00357'], #EXs : Deoxythymidine triphosphate
    ['Thymidine','cpd00184'], #EXs : In BSM
#     ['Thyminose','cpd01242'], #EXs : deoxyribose
#     ['Uracil','cpd00092'], #EXs : 
    ['Uridine','cpd00249'], #EXs : In BSM (as uracil)
#     ['XAN','cpd00309'], #EXs : Xanthine
#     ['Xanthosine','cpd01217'], #EXs : 
#     ['dATP','cpd00115'], #EXs : 
#     ['dGTP','cpd00241'], #EXs : 
#     ['dTMP','cpd00298'] #EXs : 
]

# Check to see if these metabolites are used in pathways? Should I add some of these to media? 
# Yes for ATP, and GTP. (TTP, CTP as well?)

In [4]:
# Amino Acid related metabolites
aas = [
    ['D-Alanine','cpd00117'], #EXs : 
    ['D-Glutamate','cpd00186'], #EXs : 
    ['D-Methionine','cpd00637'], #EXs : 
    ['D-Serine','cpd00550'], #EXs : 
    ['Glycine','cpd00033'], #EXs : 1
    ['L-Alanine','cpd00035'], #EXs : 2
    ['L-Arginine','cpd00051'], #EXs : 3
    ['L-Asparagine','cpd00132'], #EXs : 4
    ['L-Aspartate','cpd00041'], #EXs : 5

    ['L-Cysteine','cpd00084'], #EXs : 7
    ['L-Glutamate','cpd00023'], #EXs : 8
    ['L-Glutamine','cpd00053'], #EXs : 9
    ['L-Histidine','cpd00119'], #EXs : 10
    ['L-Isoleucine','cpd00322'], #EXs : 11
    ['L-Leucine','cpd00107'], #EXs : 12
    ['L-Lysine','cpd00039'], #EXs : 13
    ['L-Methionine','cpd00060'], #EXs : 14
    ['L-Phenylalanine','cpd00066'], #EXs : 15
    ['L-Proline','cpd00129'], #EXs : 16
    ['L-Serine','cpd00054'], #EXs : 17
    ['L-Threonine','cpd00161'], #EXs : 18
    ['L-Tryptophan','cpd00065'], #EXs : 19
    ['L-Tyrosine','cpd00069'], #EXs : 20
    ['L-Valine','cpd00156'] #EXs : 21
]
# Explore leave one out with core amino acids. 

# Dimers, and other amino acid related mets
aa_related = [
    ['2-Oxoglutarate','cpd00024'], #EXs : 
    ['Ala-Gln','cpd11587'], #EXs : 
    ['Ala-His','cpd11584'], #EXs : 
    ['Ala-Leu','cpd11583'], #EXs : 
    ['ala-L-asp-L','cpd11593'], #EXs : 
    ['ala-L-glu-L','cpd11586'], #EXs : 
    ['ala-L-Thr-L','cpd11582'], #EXs : 
    ['Aminoethanol','cpd00162'], #EXs : Ethanolamine
    ['Carnitine','cpd00266'], #EXs : 
    ['Chorismate','cpd00216'], #EXs : 
    ['L-Cysteate','cpd00395'], #EXs : 
    ['Cys-Gly','cpd01017'], #EXs : 
    ['Gly-Cys','cpd15603'], #EXs : 
    ['Gly-Gln','cpd11580'], #EXs : 
    ['Gly-Leu','cpd15604'], #EXs : 
    ['Gly-Met','cpd11591'], #EXs : 
    ['Gly-Phe','cpd15605'], #EXs : 
    ['Gly-Tyr','cpd15606'], #EXs : 
    ['gly-asn-L','cpd11581'], #EXs : 
    ['gly-asp-L','cpd11589'], #EXs : 
    ['gly-glu-L','cpd11592'], #EXs : 
    ['gly-pro-L','cpd11588'], #EXs : 
    ['L-Methionine S-oxide','cpd01914'], #EXs :
    ['L-alanylglycine','cpd11585'], #EXs : 
    ['L-methionine R-oxide','cpd11576'], #EXs : 
    ['met-L-ala-L','cpd11590'], #EXs :
    ['S-Adenosyl-L-methionine','cpd00017'], #EXs : 
    ['S-Methyl-L-methionine','cpd02027'], #EXs : 
    ['S-Ribosylhomocysteine','cpd02227'], #EXs : 
    ['N-Acetyl-D-glucosamine','cpd00122'], #EXs : 
    ['N-Acetyl-D-mannosamine','cpd00492'], #EXs : 
    ['Ornithine','cpd00064'], #EXs : 
    ['Putrescine','cpd00118'], #EXs : 
    ['Taurine','cpd00210'], #EXs : 
    ['meso-2,6-Diaminopimelate','cpd00516'] #EXs : related to lysine
]

In [6]:
t = time.time()

sys.stdout.write('Loading Models...')
universal = cobra.io.load_json_model('universal.json')
model = cobra.io.load_json_model('model.json')
bag = cobra.io.load_json_model('bag.json')
genome_id = '220668.9'
likelihoods = pickle.load(open('../likelihoods/'+ genome_id +'.probs'))
model.solver.configuration.verbosity = 3
# bag.solver.configuration.verbosity = 3

counter = 0

total_dataset_dict = {}
carb_idx = 0
nit_idx = 0
product_idx = 0
carbon = 'D-Glucose'
nitrogen = 'NH3'

### Double For-loop to set media

# Create specific Media List
media_list = bsm + M9_sources + rna_bases # + nitrogen + carbon
set_media(model, media_list, universal, verbose=False)
set_media(universal, media_list, universal, verbose=False)

# Run through each amino acid to check for production
aa_like = {}
sys.stdout.write('Starting Loop...')
for aa_list in aas:
    
    sys.stdout.write('\n'+ 'Loop' + str(counter) + ' ')
    aa = aa_list[1]+'_c'
    product = aa_list[0]
    
#     task_lb = 100
    obj = 'DM_'+ aa
    universal.objective = universal.reactions.get_by_id(obj)
    
    # Optimize universal with FBA to get possible reactions
    sys.stdout.write('Optimizing...')
    solution = universal.optimize()
    sys.stdout.write(str(round(universal.slim_optimize())) + '...')
    df = solution.fluxes.to_frame()
    active = df.loc[(abs(df['fluxes'])) > 0.1]
    
    # Add solution space to bag
    sys.stdout.write('Add to bag...')
    rxns_to_add_to_bag = []
    for rxn in active.index:
        if rxn not in [reaction.id for reaction in bag.reactions]:
            rxns_to_add_to_bag.append(universal.reactions.get_by_id(rxn))
    rxns_to_add_to_bag_copy = deepcopy(rxns_to_add_to_bag)
    
    bag.add_reactions(rxns_to_add_to_bag_copy)
    
    sys.stdout.write('Bag size: '+ str(len(bag.reactions)) +'...')
    
    # Add Demand Reaction for metabolite of interest and set to be objective
    sys.stdout.write('Add demand to model...')
    if aa not in [met.id for met in model.metabolites]:
        metabolite = deepcopy(universal.metabolite.get_by_id(aa))
        model.add_metabolites([metabolite])
    else:
        metabolite = model.metabolites.get_by_id(aa)
    demand = model.add_boundary(metabolite, type='demand')
#     model.add_reactions([demand])
    model.objective = demand
    sys.stdout.write('\n' + str(model.objective.expression) + '...')
    
    # Gapfill with probanno
    sys.stdout.write('\nPenalties...')
    # model.solver = 'gurobi'
#     gaps_to_fill = probabilistic_gapfill(model, bag, likelihoods, dm_rxns=False, ex_rxns=False)
#     gaps_to_fill = gapfill(model, bag, demand_reactions=False) # Update to probannopy gapfill function; use Gurobi
    
    with bag as bag:
        default_penalties = {'Universal': 1, 'Exchange': 100, 'Demand': 1, 'Reverse': 75}
        penalties = default_penalties
        reactions_to_remove = []
        for r in bag.reactions:
            if model.reactions.has_id(r.id):
                reactions_to_remove.append(r)
                penalties[r.id] = 0  # In the model
            elif r.id in likelihoods:
                penalties[r.id] = max(0, 1 - likelihoods[r.id]) * (penalties[r.id] if r.id in penalties else 1)
            else:
                penalties[r.id] = 1
#         bag.remove_reactions(reactions_to_remove)
        sys.stdout.write('Gapfilling...')
        dont_continue = 0
        try:
            gaps_to_fill = gapfill(model, bag, penalties=penalties, demand_reactions=False)
        except:
            try:
                gaps_to_fill = gapfill(model, bag, penalties=penalties, demand_reactions=False)
            except:
                dont_continue = 1
                pass
    
    if dont_continue == 0:
        # Fill the gaps
        rxns_to_add = []
        for gap in gaps_to_fill:
            model.add_reactions(gap)

        # Optimize with filled pathway
        sys.stdout.write('pFBA...')
        solution = pfba(model, objective = demand)
        sys.stdout.write(str(round(model.slim_optimize())) + '...')

        sys.stdout.write('Constructing Dict...')
        # Find reactions that carry flux
        df = solution.fluxes.to_frame()
        active = df.loc[(abs(df['fluxes'])) > 0.1]

        # Acquire likelihood scores for reactions that carry flux
        flux_rxns = []
        like_list = []
        for rxn in list(active.index):
            if rxn.startswith('rxn'):
                try:
                    flux_rxns.append([str(rxn),likelihoods[str(rxn)]])
                    like_list.append(likelihoods[str(rxn)])
                except:
                    pass
        avg_like = np.mean(like_list)
        sys.stdout.write('Ave likelihood of: ' + aa + ' is ' + str(avg_like))

        counter += 1

        report_dict = {}

        report_dict['Model_ID'] = genome_id
        report_dict['Carbon'] = carbon
        report_dict['Nitrogen'] = nitrogen
        report_dict['objective'] = product
        report_dict['avg_path_like'] = avg_like
        report_dict['gaps_filled'] = gaps_to_fill[0]
        report_dict['reactions_w_flux'] = flux_rxns
        report_dict['active_rxns'] = active

        report_dict_ID = genome_id + ':' + str(carb_idx) + '.' + str(nit_idx) + '.' + str(product_idx)
        total_dataset_dict[report_dict_ID] = report_dict
        product_idx += 1 #Keep track to which product is being maximized

        # Remove reactions to reset
        sys.stdout.write('Resetting...')
        model.remove_reactions([demand])
        model.remove_reactions(gaps_to_fill[0])
        bag.remove_reactions(rxns_to_add_to_bag_copy)

        elapsed = time.time() - t
        sys.stdout.write('Run time: ' + str(elapsed/60) + " [mins]")

    elif dont_continue == 1:
        sys.stdout.write('Failed to gapfill...')
        counter += 1

        report_dict = {}

        report_dict['Model_ID'] = genome_id
        report_dict['Carbon'] = carbon
        report_dict['Nitrogen'] = nitrogen
        report_dict['objective'] = product
        report_dict['avg_path_like'] = "Failed to gapfill"
        report_dict['gaps_filled'] = "Failed to gapfill"
        report_dict['reactions_w_flux'] = "Failed to gapfill"
        report_dict['active_rxns'] = "Failed to gapfill"

        report_dict_ID = genome_id + ':' + str(carb_idx) + '.' + str(nit_idx) + '.' + str(product_idx)
        total_dataset_dict[report_dict_ID] = report_dict
        product_idx += 1 #Keep track to which product is being maximized

        # Remove reactions to reset
        sys.stdout.write('Resetting...')
        model.remove_reactions([demand])
        bag.remove_reactions(rxns_to_add_to_bag_copy)

        elapsed = time.time() - t
        sys.stdout.write('Run time: ' + str(elapsed/60) + " [mins]")
        
        
file_name = "../metabolic_output/%s.data" % (genome_id)
pickle.dump(total_dataset_dict, open(file_name, "wb"))

elapsed = time.time() - t
print("\nTime to complete: " + str(elapsed/60) + " [mins]")

Loading Models...Starting Loop...
Loop0 Optimizing...1000.0...Add to bag...Bag size: 1104...Add demand to model...
1.0*DM_cpd00117_c - 1.0*DM_cpd00117_c_reverse_21bac...
Penalties...Gapfilling...pFBA...1000.0...Constructing Dict...Ave likelihood of: cpd00117_c is 0.4436602411303152Resetting...Run time: 0.722484548887 [mins]
Loop1 Optimizing...1000.0...Add to bag...Bag size: 1095...Add demand to model...
1.0*DM_cpd00186_c - 1.0*DM_cpd00186_c_reverse_17bfd...
Penalties...Gapfilling...pFBA...250.0...Constructing Dict...Ave likelihood of: cpd00186_c is 0.4647303441437369Resetting...Run time: 2.13949328264 [mins]
Loop2 Optimizing...1000.0...Add to bag...Bag size: 1049...Add demand to model...
1.0*DM_cpd00637_c - 1.0*DM_cpd00637_c_reverse_77fb4...
Penalties...Gapfilling...pFBA...118.0...Constructing Dict...Ave likelihood of: cpd00637_c is 0.4599532806600122Resetting...Run time: 364.687907732 [mins]
Loop3 Optimizing...1000.0...Add to bag...Bag size: 1103...Add demand to model...
1.0*DM_cpd005

In [7]:
total_dataset_dict


{'220668.9:0.0.0': {'Carbon': 'D-Glucose',
  'Model_ID': '220668.9',
  'Nitrogen': 'NH3',
  'active_rxns':                fluxes
  rxn00545_c      500.0
  rxn00148_c    -1000.0
  rxn01102_c      625.0
  rxn00781_c      375.0
  rxn05319_c     1000.0
  rxn01492_c     -500.0
  rxn05466_c     1000.0
  rxn00747_c     -375.0
  rxn05573_c      500.0
  rxn01013_c     1000.0
  rxn01100_c     -375.0
  rxn00611_c     -125.0
  rxn01106_c    -1000.0
  rxn10042_c     -375.0
  rxn00184_c    -1000.0
  rxn00459_c     1000.0
  rxn00615_c     -125.0
  rxn00216_c      500.0
  rxn00558_c      500.0
  rxn00763_c      125.0
  rxn01011_c    -1000.0
  rxn00193_c     1000.0
  rxn01286_c      625.0
  EX_cpd00067_e  1000.0
  EX_cpd00001_e  1000.0
  EX_cpd00013_e -1000.0
  EX_cpd00027_e  -500.0
  DM_cpd00117_c  1000.0
  rxn01870_c      500.0
  rxn00849_c    -1000.0,
  'avg_path_like': 0.4436602411303152,
  'gaps_filled': [<Reaction rxn01870_c at 0x7f7724f61250>,
   <Reaction rxn00849_c at 0x7f7724f53610>],
  'obje

In [8]:
file_name = "../metabolic_output/%s_prob_gapfill_test.data" % (genome_id)
pickle.dump(total_dataset_dict, open(file_name, "wb"))

In [ ]:
# Using pFBA with new media components (RNA bases + thymidine...) 
# Remove reaction likelihoods of zero from model
# Add demands for all metabolites in model to avoid any reactions being blocked
# Use thresholded likelihoods to penalize reactions with no likelihood using pFBA gapfill
# Use pFBA answer + 0 likelihood reactions from reconstructed model + higher likelihood reactions to probanno gapfill

t = time.time()
counter = 0

sys.stdout.write('Loading in models...')

universal = cobra.io.load_json_model("../Data/GramPosUni.json")
genome_id = '220668.9'
model = cobra.io.read_sbml_model('../gap_models/'+ genome_id +'.xml')
likelihoods = pickle.load(open('../likelihoods/'+ genome_id +'.probs'))

sys.stdout.write('Adding Water...')

# Ensure free diffusion of water
model.reactions.get_by_id('rxn05319_c').name = "Water transport"
model.reactions.get_by_id('rxn05319_c').bounds = (-1000., 1000.)

sys.stdout.write('Set-up Universal...')

### Set up Universal
# Add all model reactions to Universal model
model_reactions = copy.deepcopy(model.reactions)
model_reactions_to_remove = []
for rxn in model.reactions:
    if rxn.id in set([reaction.id for reaction in universal.reactions]):
        model_reactions_to_remove.append(rxn)
universal.remove_reactions([reaction.id for reaction in model_reactions_to_remove])
universal.add_reactions(model_reactions)

# Add GPRs from likelihood dict to universal reactions that are still missing them
rxn_ids = [reaction.id for reaction in universal.reactions]
rxns_to_fix = []
for rxn in rxn_ids:
    if rxn.startswith('rxn'):
        try: # This catches all of the reactions that have no likelihood value.
            if likelihoods[rxn] > 0.0:
                if universal.reactions.get_by_id(rxn).gene_reaction_rule == '':
                    if likelihoods.data[rxn]['gpr'] != '':
                        rxns_to_fix.append(rxn)
                        universal.reactions.get_by_id(rxn).gene_reaction_rule = likelihoods.data[rxn]['gpr']
        except:
            pass

# Add demand for all metabolites in Universal model to stop blocked reactions
all_mets = []
for met in universal.metabolites:
    if (met.id.endswith('_c')):
        universal.add_boundary(met, type='demand')

### Set Up Model: remove low likelihood reactions
sys.stdout.write('Set-up Model...')
low_like_model = []
for rxn in model.reactions:
    if rxn.id.startswith('rxn'):
        try:
            if likelihoods[rxn.id] <= 0.1:
                low_like_model.append(rxn.id)
        except:
            pass
model_rxns_to_remove = [model.reactions.get_by_id(rxn) for rxn in low_like_model]
model.remove_reactions(model_rxns_to_remove)

### Set Up Bag: Leave only low-likelihood-model reactions, and high-likelihood-non-model reactions for now
sys.stdout.write('Set-up Bag...')

# Make deepcopy of universal for bag to process later
bag = copy.deepcopy(universal)

# Find reaction IDs for the high-likelihood reactions only in the universal model
high_like_non_model = []
for rxn in universal.reactions:
    if rxn.id.startswith('rxn') and rxn.id not in [reaction.id for reaction in model.reactions]:
        try:
            if likelihoods[rxn.id] >= 0.1:
                high_like_non_model.append(rxn.id)
        except:
            pass

uni_rxn_ids = [reaction.id for reaction in universal.reactions]
rxns_to_remove = set(uni_rxn_ids).difference(set(high_like_non_model).union(set(low_like_model)))

sys.stdout.write('Trimming Bag...')
bag.remove_reactions([bag.reactions.get_by_id(rxn) for rxn in rxns_to_remove])

# Save Models
sys.stdout.write('Saving...')
cobra.io.save_json_model(universal, "universal.json")
cobra.io.save_json_model(model, "model.json")
cobra.io.save_json_model(bag, "bag.json")

elapsed = time.time() - t
print("\nTime to complete: " + str(elapsed/60) + " [mins]")